### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [1]:
%%capture
!pip install transformers

In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP/WEEK4/예습과제1/bbc-text.csv') # bbc-text.csv 파일 경로

In [5]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
print(len(df))

2225


In [7]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [8]:
# BertTokenizer를 사용하여 텍스트를 BERT 모델에 맞는 형식으로 토큰화 + 뉴스 카테고리 레이블링
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [9]:
# 텍스트 데이터를 BERT 모델에 맞게 처리하는 커스텀 데이터셋 클래스
class Dataset(torch.utils.data.Dataset):


    def __init__(self, df):
        # df['category']에서 각 텍스트의 카테고리 이름을 가져와 labels 딕셔너리에서 숫자로 매핑
        self.labels = [labels[label] for label in df['category']]
        # 각 텍스트 데이터를 BERT 토크나이저를 사용하여 BERT 모델에 맞게 토큰화
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        # 클래스 레이블인 self.labels를 반환 -> 데이터셋에 있는 모든 레이블을 가져옴.
        return self.labels

    def __len__(self):
        # 데이터셋의 길이를 반환
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [10]:
# BERT 모델을 기반으로 텍스트 분류를 위한 신경망 모델
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # BERT 모델 불러오기 (bert-base-cased: 대소문자를 구분하는 모델)
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        # BERT 모델에 input_id와 attention_mask를 넣어서 처리
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)

        # 선형 변환을 통해 5개의 클래스에 대해 점수 계산
        linear_output = self.linear(dropout_output)
        # ReLU 활성화 함수 적용
        final_layer = self.relu(linear_output)

        return final_layer

In [11]:
# BERT 기반 텍스트 분류 모델을 학습
def train(model, train_data, val_data, learning_rate, epochs):

    # 데이터셋 준비
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # criterion은 교차 엔트로피 손실 함수
    criterion = nn.CrossEntropyLoss()
    # optimizer는 Adam 옵티마이저
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            # tqdm은 진행 바를 출력해 주는 라이브러리로, 학습 진행 상황을 시각적으로 확인 가능
            for train_input, train_label in tqdm(train_dataloader):

                # train_label: 배치의 정답 레이블
                train_label = train_label.to(device)
                # mask: 입력 문장의 attention mask입니다. 패딩된 부분을 무시하기 위한 마스크
                mask = train_input['attention_mask'].to(device)
                # input_id: input_ids는 텍스트를 숫자로 변환한 ID 값
                input_id = train_input['input_ids'].squeeze(1).to(device)

                # 모델에 입력 텍스트와 attention mask를 넣고 예측값을 반환
                output = model(input_id, mask)

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # 모델의 기울기를 초기화하여, 이전 배치에서의 기울기가 남아있는 것을 방지
                model.zero_grad()
                # 손실에 대해 역전파(backpropagation)를 수행하여 모델의 가중치 기울기를 계산
                batch_loss.backward()
                # 옵티마이저가 기울기를 바탕으로 가중치를 업데이트
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            # 검증 과정
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [12]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [13]:
np.random.seed(112)
# frac=1은 전체 데이터를 섞겠다는 의미
# 무작위로 섞인 데이터를 세 부분으로 나누는데, 첫 번째 분할 지점은 int(.8*len(df))로 전체 데이터의 80% 위치에 해당하며, 두 번째 분할 지점은 int(.9*len(df))로 90% 위치에 해당
# df_train: 데이터의 80%는 훈련 데이터 / df_val: 데이터의 10%는 검증 데이터 / df_test: 나머지 10%는 테스트 데이터
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

100%|██████████| 890/890 [03:05<00:00,  4.79it/s]


Epochs: 1 | Train Loss:  0.754 | Train Accuracy:  0.354 | Val Loss:  0.580 | Val Accuracy:  0.689


100%|██████████| 890/890 [03:06<00:00,  4.78it/s]


Epochs: 2 | Train Loss:  0.335 | Train Accuracy:  0.913 | Val Loss:  0.176 | Val Accuracy:  0.982


In [15]:
evaluate(model, df_test)

Test Accuracy:  0.982


optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗